# Open AI Basics

## Lerninhalte

- Nutzung eines OpenAI API-Keys
- Kennen der OpenAI API
- Kennen der OpenAI Python-Bibliothek
- Senden eines dynamischen Prompts an die OpenAI API und Ausgabe der Antwort

## Anwendungsfall

META: Kategorisierung und Auswahl der richtigen Kategorie.
Fall: Ein Kunde möchte eine Bestellung aufgeben, aber nicht direkt eine Pizza bestellen, sondern eine Zutat nennen. Die Aufgabe besteht darin, die passende Kategorie zu finden.

## Setup

Zunächst müssen wir die OpenAI Python-Bibliothek installieren. Dazu installieren wir gleich ein paar mehr Bibliotheken,
welche uns später das Programmieren leichter machen. Dies kann über den folgenden Befehl erfolgen:

In [10]:
#Wird auf der Konsole ausgeführt
%pip install json openai requests

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
import os.path
import requests

MENU_URL = ""

response = requests.get(MENU_URL)
with open(os.path.basename(MENU_URL), 'w', encoding='utf-8') as f:
    f.write(response.text)

from menu import MENU

In [ ]:
from openai import OpenAI

from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')    # Alternative 1: Google Colab
#openai_api_key = os.getenv('OPENAI_API_KEY')      # Alternative 2: Lokaler Systemumgebungsvariable, ggf. mit .env-Datei
#openai_api_key = "your_openai_api_key_here"       # Alternative 3: Direkt im Code (nicht empfohlen)

openai_client = OpenAI(api_key = openai_api_key)

In [18]:
import json

user_input = "Ich möchte gerne etwas mit scharfer Salami oder Pilzen."

prompt = f"Speisekarte:\n{json.dumps(MENU, ensure_ascii=False)}\n\nBestellung: {user_input}"

Hier sind die passenden Optionen für dich:

1. **Pizza Salami**  
   *Beschreibung:* Tomatensauce, Mozzarella, scharfe Salami  
   *Preis:* 9.5 €

2. **Pizza Diavola**  
   *Beschreibung:* Scharfe Salami, Chili, Paprika  
   *Preis:* 10.0 €

3. **Pizza Funghi**  
   *Beschreibung:* Tomatensauce, Mozzarella, frische Champignons  
   *Preis:* 9.0 €

4. **Pizza Capricciosa**  
   *Beschreibung:* Schinken, Pilze, Artischocken, Oliven  
   *Preis:* 10.5 €

5. **Pizza Champignon**  
   *Beschreibung:* Extra Pilze, Mozzarella  
   *Preis:* 9.0 €

6. **Pizza Tartufo**  
   *Beschreibung:* Trüffelcreme, Pilze  
   *Preis:* 12.0 €

Möchtest du eine dieser Pizzen bestellen?


In [21]:
# Unstrukturierte Ausgabe

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."
        },
        {
            "role": "user",
            "content":  prompt
		}
    ],
)

print(response.output_text)

Hier sind zwei passende Optionen für Sie:

1. **Pizza Salami**
   - **Beschreibung**: Tomatensauce, Mozzarella, scharfe Salami
   - **Preis**: 9.5 €

2. **Pizza Funghi**
   - **Beschreibung**: Tomatensauce, Mozzarella, frische Champignons
   - **Preis**: 9.0 €

Möchten Sie eine der beiden Pizzen bestellen?


In [22]:
# Strukturierte Ausgabe

expected_response_schema = {
    "type": "object",
    "properties": {
        "items": {
            "type": "array",  #Erwarteter Rückgabetyp ist ein Array von Objekten
            "items": {
                "type": "object",
                "properties": {
                    "id": {"type": "string"},
                    "name": {"type": "string"},
                    "description": {"type": "string"},
                    "category": {"type": "string"},
                    "price": {"type": "number"},
                },
                "required": ["id", "name", "description", "category", "price"],
                "additionalProperties": False,
            },
        }
    },
    "required": ["items"],
    "additionalProperties": False,
}

response = openai_client.responses.create(
    model = "gpt-4o",
    input = [
        {
            "role": "developer",
            "content": "Du bist ein digitaler Kellner. Wähle den passenden Eintrag aus der Speisekarte aus."
        },
        {
            "role": "user",
            "content":  prompt
		}
    ],
    # Gewünschtes Ausgabeformat wird API-Aufruf übergeben
    text={
        "format": {
            "type": "json_schema",
            "name": "menu_item",
            "schema": expected_response_schema,
            "strict": True
        }
    }
)
recommendations = json.loads(response.output_text)["items"]

for item in recommendations:
    print(item)


{'id': 'pizza_salami', 'name': 'Pizza Salami', 'description': 'Tomatensauce, Mozzarella, scharfe Salami', 'category': 'Pizza', 'price': 9.5}
{'id': 'pizza_diavola', 'name': 'Pizza Diavola', 'description': 'Scharfe Salami, Chili, Paprika', 'category': 'Pizza', 'price': 10.0}
{'id': 'pizza_funghi', 'name': 'Pizza Funghi', 'description': 'Tomatensauce, Mozzarella, frische Champignons', 'category': 'Pizza', 'price': 9.0}
{'id': 'pizza_capricciosa', 'name': 'Pizza Capricciosa', 'description': 'Schinken, Pilze, Artischocken, Oliven', 'category': 'Pizza', 'price': 10.5}
{'id': 'pizza_champignon', 'name': 'Pizza Champignon', 'description': 'Extra Pilze, Mozzarella', 'category': 'Pizza', 'price': 9.0}
